In [2]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
import matplotlib.pyplot as plt
import numpy as np
from qiskit import transpile
from qiskit.visualization import plot_histogram
import bluequbit
from qiskit_aer import Aer
from qiskit import QuantumCircuit
import networkx as nx

backend = Aer.get_backend('aer_simulator')
bq = bluequbit.init("GTLeS22Wxwhod5iYTXyQtzPVao1PuWBX")

[BQ-PYTHON-SDK][WARNING] - Beta version 0.12.0b1 of BlueQubit Python SDK is being used.


## Functions

In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit.converters import circuit_to_dag, dag_to_circuit

# Convert to DAG to access layers
def cut_circuit(quan_circ, fraction_depth):
    dag = circuit_to_dag(quan_circ)

    # Get the layers (each is a dict with 'graph' key)
    layers = list(dag.layers())

    # Compute split point based on number of layers/fraction_depth
    split_point = len(layers) // fraction_depth

    # Create new DAGs for each half
    dag1 = dag.__class__()
    dag2 = dag.__class__()

    # Add same registers to new DAGs
    for qreg in dag.qregs.values():
        dag1.add_qreg(qreg)
        dag2.add_qreg(qreg)
    for creg in dag.cregs.values():
        dag1.add_creg(creg)
        dag2.add_creg(creg)

    # Add layers to each new DAG
    for i, layer in enumerate(layers):
        if i < split_point:
            dag1.compose(layer['graph'])
        else:
            dag2.compose(layer['graph'])

    # Convert DAGs back to circuits
    qc = dag_to_circuit(dag1)
    return qc

def get_connected_components(circuit):
    # This will 
    G = nx.Graph()
    qubits = list(circuit.qubits)
    
    # Add all qubits as nodes
    for qubit in qubits:
        G.add_node(qubit)

    # Go through each instruction in the circuit
    for instr, qargs, _ in circuit.data:
        if len(qargs) > 1:  # Multi-qubit gate
            for i in range(len(qargs)):
                for j in range(i + 1, len(qargs)):
                    G.add_edge(qargs[i], qargs[j])

    # Get connected components
    return list(nx.connected_components(G))


## Circuit cutting vertically

In [ ]:
# load the circuit from a .qasm file
qc4 =  QuantumCircuit.from_qasm_file('P4_golden_mountain.qasm')
# cut the circuit based on the second the parameter
sim_circ = cut_circuit(qc4, 6)

# run simulations
result = bq.run(sim_circ, device='mps.cpu', shots=1000)
counts = result.get_counts()
print("Raw counts:", counts)

peak_bitstring = max(counts, key=counts.get)

peak_bitstring_reversed = peak_bitstring[::-1]

print("Peak bitstring:     ", peak_bitstring)
print("Reversed bitstring: ", peak_bitstring_reversed)

[BQ-PYTHON-SDK][INFO] - Submitted: Job ID: EhtTbfyQXMoGBd1j, device: mps.cpu, run status: RUNNING, created on: 2025-04-13 04:58:38 UTC, estimated runtime: 100 ms, estimated cost: $0.00, num qubits: 48
Raw counts: {'000000000010010011001101111000110111001001101101': 1, '000000001001101011100000111101110101011111100101': 1, '000000001001101110110110010110111111010101110000': 1, '000000010111100010010111001111100011011000010001': 1, '000000010111110101101111111110110101011010000010': 1, '000000011001100011011011101011101011001000000010': 1, '000000011111100101100111100110001110110001000110': 1, '000000100011000001011000000111111110000000100100': 1, '000000100110100001011100000001001010110110110010': 1, '000000100110100011100100010001100111000101011010': 1, '000000101000010001110000001110101010100100110001': 1, '000000101000010010011010101100101110001111010101': 1, '000000101000010110001101010000000000100100000010': 1, '000000101101001110110110000010010111111010010000': 1, '000000101101011

## Check Connectivity

In [6]:
# check the connectivity of the circuit
qc1 =  QuantumCircuit.from_qasm_file('P1_little_peak.qasm')
qc2 =  QuantumCircuit.from_qasm_file('P2_swift_rise.qasm')
qc3 =  QuantumCircuit.from_qasm_file('P3__sharp_peak.qasm')
qc4 =  QuantumCircuit.from_qasm_file('P4_golden_mountain.qasm')
qc5 =  QuantumCircuit.from_qasm_file('P5_granite_summit.qasm')
qc6 =  QuantumCircuit.from_qasm_file('P6_titan_pinnacle.qasm')
qcs = [qc1,qc2,qc3,qc4,qc5,qc6]

for i,qc in enumerate(qcs):
    print(f"Connected components for problem {i+1} = ", get_connected_components(qc))

Connected components for problem 1 =  [{<Qubit register=(4, "q"), index=0>}, {<Qubit register=(4, "q"), index=1>}, {<Qubit register=(4, "q"), index=2>}, {<Qubit register=(4, "q"), index=3>}]
Connected components for problem 2 =  [{<Qubit register=(28, "q"), index=0>, <Qubit register=(28, "q"), index=9>, <Qubit register=(28, "q"), index=20>, <Qubit register=(28, "q"), index=7>, <Qubit register=(28, "q"), index=4>, <Qubit register=(28, "q"), index=19>, <Qubit register=(28, "q"), index=26>, <Qubit register=(28, "q"), index=21>, <Qubit register=(28, "q"), index=22>, <Qubit register=(28, "q"), index=18>, <Qubit register=(28, "q"), index=27>, <Qubit register=(28, "q"), index=6>, <Qubit register=(28, "q"), index=3>, <Qubit register=(28, "q"), index=8>, <Qubit register=(28, "q"), index=13>, <Qubit register=(28, "q"), index=11>, <Qubit register=(28, "q"), index=5>, <Qubit register=(28, "q"), index=17>, <Qubit register=(28, "q"), index=15>, <Qubit register=(28, "q"), index=2>, <Qubit register=(2

/var/folders/w4/whqyt1tj44x904hj4gl223gr0000gn/T/ipykernel_14852/3176131954.py:47: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 3.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  for instr, qargs, _ in circuit.data:
